In [4]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class DFS():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class BFS(DFS):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

class Maze():

    def __init__(self, filename):


        with open(filename) as f:
            contents = f.read()

        if contents.count("S") != 1:
            raise Exception("maze must have exactly one start point")
        if contents.count("G") != 1:
            raise Exception("maze must have exactly one goal")

        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j] == "S":
                        self.start = (i, j)
                        row.append(False)
                    elif contents[i][j] == "G":
                        self.goal = (i, j)
                        row.append(False)
                    elif contents[i][j] == "0":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)

        self.solution = None


    def solve_with_BFS(self):

        self.num_explored = 0


        start = Node(state=self.start, parent=None, action=None)
        frontier = BFS()
        frontier.add(start)


        self.explored = set()


        while True:


            if frontier.empty():
                raise Exception("no solution")

            node = frontier.remove()
            self.num_explored += 1

            if node.state == self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            self.explored.add(node.state)

            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)
    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]

        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result

    def solve_with_DFS(self):


        self.num_explored = 0

        start = Node(state=self.start, parent=None, action=None)
        frontier = DFS()
        frontier.add(start)

        self.explored = set()

        while True:

            if frontier.empty():
                raise Exception("no solution")

            node = frontier.remove()
            self.num_explored += 1

            if node.state == self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            self.explored.add(node.state)

            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)

    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("1", end="")
                elif (i, j) == self.start:
                    print("S", end="")
                elif (i, j) == self.goal:
                    print("G", end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print("0", end="")
            print()
        print()



m = Maze("maze.txt")
print("Maze:")
m.print()
print("Solving with BFS     :")

m.solve_with_BFS()
print("States Explored:", m.num_explored)
print("Solution:")
m.print()

print("   Solving with DFS      :")
m.solve_with_DFS()
print("   States Explored    :", m.num_explored)
print("    Solution   :")
m.print()

Maze:

10S0100
1100011
0101000
1101101
0101000
0111011
G000000

Solving with BFS     :
States Explored: 24
Solution:

10S0100
11***11
0101**0
11011*1
0101**0
0111*11
G****00

   Solving with DFS      :
   States Explored    : 19
    Solution   :

10S*100
110**11
0101**0
11011*1
0101**0
0111*11
G****00

